# Carregando pastas de arquivos tif com múltiplos canais / múltiplas posições
Alguns microscópios gravam dados de imagem em disco como arquivos tif, fatia por fatia. Você encontra então muitos arquivos nomeados, por exemplo, `image_z03_ch01.tif` nessas pastas. Para carregar essas pastas, você geralmente precisa saber detalhes como quantas fatias z e quantos canais foram imageados, ou se os dados da imagem têm múltiplos pontos de tempo ou não.

Veja também [esta discussão](https://forum.image.sc/t/reading-2d-images-as-a-multi-channel-3d-stack-in-python/75460).

In [1]:
from skimage import io
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np

import shutil
import os

path = "../../data/tif_folder/"

Para fins de demonstração, vamos criar uma pasta com dados de imagem fictícios.

In [2]:
if not os.path.exists(path):
    os.mkdir(path)

In [3]:
for z in range(1,7):
    for c in range(1,5):
        #image = io.imread("c:/structure/data/blobs.tif")
        #io.imsave(f"c:/structure/data/images/r01c01f34p0{z}-ch0{c}t01.tiff", image)
        
        shutil.copy("../../data/blobs.tif",
                    path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff")

Para ter uma visão geral, podemos imprimir os nomes dos arquivos na pasta.

In [4]:
for file in os.listdir(path):
    print(file)

image_z01-ch01.tiff
image_z01-ch02.tiff
image_z01-ch03.tiff
image_z01-ch04.tiff
image_z02-ch01.tiff
image_z02-ch02.tiff
image_z02-ch03.tiff
image_z02-ch04.tiff
image_z03-ch01.tiff
image_z03-ch02.tiff
image_z03-ch03.tiff
image_z03-ch04.tiff
image_z04-ch01.tiff
image_z04-ch02.tiff
image_z04-ch03.tiff
image_z04-ch04.tiff
image_z05-ch01.tiff
image_z05-ch02.tiff
image_z05-ch03.tiff
image_z05-ch04.tiff
image_z06-ch01.tiff
image_z06-ch02.tiff
image_z06-ch03.tiff
image_z06-ch04.tiff


O scikit-image oferece um [`imread_collection`](https://scikit-image.org/docs/stable/api/skimage.io.html#skimage.io.imread_collection) para carregar arquivos que correspondem a um padrão, por exemplo, contendo `*`.

In [5]:
im_collection = io.imread_collection(path + "*")
im_collection

Você pode transformar essa coleção de imagens em uma pilha de imagens baseada em array numpy. Infelizmente, o número de fatias z e canais é desconhecido neste ponto.

In [6]:
image_stack = im_collection.concatenate()
image_stack.shape

(24, 254, 256)

Se você conhece o número de fatias z e canais, pode [`reshape`]() a imagem para uma imagem 3D+ch ou 4D.

In [7]:
num_channels = 4
num_z_slices = 6
image4d = np.reshape(image_stack, (num_channels, num_z_slices, image_stack.shape[-2], image_stack.shape[-1]))
image4d.shape

(4, 6, 254, 256)

Alternativamente, você também pode criar seus próprios loops for para carregar os dados do disco. Isso lhe dá um pouco mais de liberdade, por exemplo, para ordenar fatias e canais nas dimensões usadas.

In [8]:
num_channels = 4
num_z_slices = 6

image4d_loaded = np.asarray([
    [io.imread(path + f"image_z{str(z).zfill(2)}-ch{str(c).zfill(2)}.tiff") for c in range(1, 5)]
    for z in range(1, 7)
])
image4d_loaded.shape

(6, 4, 254, 256)